In [2]:
from typing import List, Dict
import requests
from bs4 import BeautifulSoup
from collections import Counter
import cv2
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial import distance
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Guilherme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def read_image_colors(image_path):
    altura, largura = 250, 250
    imagem = cv2.imread(image_path)
    imagem = cv2.resize(imagem, (largura, altura))
    imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

    cores_encontradas = []
    for linha in imagem_rgb:
        for pixel in linha:
            cor = tuple(pixel)
            cores_encontradas.append(cor)

    return cores_encontradas


imagem_path = r'D:\testekeras\cozinha.jpg'


cores_da_imagem = read_image_colors(imagem_path)


dados = pd.read_csv(r"D:\testekeras\all_cores.csv", sep=";", encoding="latin-1")


def converter_codigo_rgb(valor):
    try:
        valor = valor.replace('RGB', '').strip()
        valores = tuple(map(int, valor.split(',')))
        return valores
    except ValueError:
        return None

dados['CodigoRgb'] = dados['CodigoRgb'].apply(converter_codigo_rgb)



dados = dados.dropna(subset=['CodigoRgb'])


num_clusters = 5


cores_imagem_array = np.array(cores_da_imagem)


kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(cores_imagem_array)


centroides = kmeans.cluster_centers_


def encontrar_cor_mais_proxima(cor, cores_df):
    distancias = [distance.euclidean(cor, c) for c in cores_df]
    indice_cor_mais_proxima = np.argmin(distancias)
    return cores_df[indice_cor_mais_proxima]


cores_proximas = [encontrar_cor_mais_proxima(centro, dados['CodigoRgb'].to_list()) for centro in centroides]


for i, cor_proxima in enumerate(cores_proximas):
    cluster = i + 1
    cor_dataframe = tuple(cor_proxima)
    nome_cor_dataframe = dados[dados['CodigoRgb'].apply(lambda x: tuple(x)) == cor_dataframe]['CorTraduzida'].values[0]
    print(f'Cluster {cluster}: Cor do Cluster: {cor_proxima}, Cor do DataFrame: {cor_dataframe}, Nome da Cor: {nome_cor_dataframe}')

In [3]:

def extract_keywords(url: str) -> List[str]:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text()
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    keyword_counter = Counter(filtered_words)
    keywords = [word for word, count in keyword_counter.most_common(10)]
    return keywords


def train_text_analysis_model(data: List[str], labels: List[int], max_words: int, embedding_dim: int, max_length: int, num_epochs: int) -> tf.keras.models.Sequential:
    tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, truncating='post')
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(padded_sequences, labels, epochs=num_epochs)

    return model

lista_urls_concorrente = [
    "https://www.flooranddecor.com/ledgers-stone/alamo-sandstone-random-ledger-panel-100899772.html",
    "https://www.flooranddecor.com/porcelain-ceramic-decoratives/zellige-pearl-opal-polished-ceramic-tile-100776673.html",
    "https://www.flooranddecor.com/limestone-basalt-stone/black-brushed-limestone-tile-100046804.html",
    "https://www.flooranddecor.com/limestone-basalt-stone/basalt-3-in.-hexagon-mosaic-100654334.html"
]

lista_urls_portobello = ["https://www.portobello.com.br/produtos/chelsea/acessorios-revestimento/chelsea-dark-red/14x26-natural/26794ET"
 ,"https://www.portobello.com.br/produtos/chelsea/chelsea-off-white/1418/revestimentos/acessorios",
 "https://www.portobello.com.br/produtos/chelsea/chelsea-grey/3685/revestimentos/acessorios",
 "https://www.portobello.com.br/produtos/chelsea/chelsea-dark-red/1419/revestimentos/acessorios",
 "https://www.portobello.com.br/produtos/easy-collection/field-tile/granville-gold/80x80-externo/200368E?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/brisa/4569/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/fossil-grey/4574/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/granville-sand/4572/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/harbour/4570/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/harbour/4570/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/easy-collection/granville-gold/4573/revestimentos/porcelanato?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/metallica/pecas-de-acabamento/met-line-white/03x120-mate/201880ET?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/metallica/met-line-black/1651/revestimentos/pecas-de-acabamento?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/metallica/met-line-corten/7832/revestimentos/pecas-de-acabamento?channel=portobello-shop"
 ,"https://www.portobello.com.br/produtos/metallica/met-line-gold/1659/revestimentos/pecas-de-acabamento?channel=portobello-shop",
 "https://www.portobello.com.br/produtos/steel/field-tile/steel-black/60x120-mate/27798E?channel=portobello-shop",
 "https://www.portobello.com.br/produtos/steel/field-tile/steel-corten/60x120-mate/22593E?channel=portobello-shop"]


"""Extraindo keywords concorrência"""
concorrente_keywords: Dict[str, List[str]] = {}
for url in lista_urls_concorrente:
    keywords = extract_keywords(url)
    concorrente_keywords[url] = keywords
    print(f"Palavras-chave para {url}: {keywords}")


"""Extraindo keywords Portobello"""
portobello_keywords: Dict[str, List[str]] = {}
for url in lista_urls_portobello:
    keywords = extract_keywords(url)
    portobello_keywords[url] = keywords
    print(f"Palavras-chave para {url}: {keywords}")


"""TReinar modelo com palavras chaves"""
"""Adicionar futuramente modelo com imagens e identificação de codigos hexadecimal de cores"""
data = [" ".join(keywords) for keywords in concorrente_keywords.values()]
labels = np.array([1] * len(data)) 
max_words = 1000
embedding_dim = 100
max_length = 100
num_epochs = 10

model = train_text_analysis_model(data, labels, max_words, embedding_dim, max_length, num_epochs)


"""Compara palavras chaves entre Concorrente e Portobello"""
for url_concorrente, keywords_concorrente in concorrente_keywords.items():
    for url_portobello, keywords_portobello in portobello_keywords.items():
        overlap = set(keywords_concorrente) & set(keywords_portobello)
        similarity = len(overlap) / max(len(keywords_concorrente), len(keywords_portobello))
        print(f"Similaridade entre {url_concorrente} e {url_portobello}: {similarity:.2%}")

Palavras-chave para https://www.flooranddecor.com/ledgers-stone/alamo-sandstone-random-ledger-panel-100899772.html: ['shop', 'projects', 'pro', 'add', 'services', 'design', 'home', 'installation', 'piece', 'back']
Palavras-chave para https://www.flooranddecor.com/porcelain-ceramic-decoratives/zellige-pearl-opal-polished-ceramic-tile-100776673.html: ['shop', 'projects', 'pro', 'add', 'tile', 'services', 'design', 'installation', 'home', 'back']
Palavras-chave para https://www.flooranddecor.com/limestone-basalt-stone/black-brushed-limestone-tile-100046804.html: ['shop', 'projects', 'pro', 'add', 'stone', 'services', 'design', 'home', 'installation', 'tile']
Palavras-chave para https://www.flooranddecor.com/limestone-basalt-stone/basalt-3-in.-hexagon-mosaic-100654334.html: ['shop', 'projects', 'add', 'pro', 'design', 'services', 'installation', 'home', 'piece', 'stone']
Palavras-chave para https://www.portobello.com.br/produtos/chelsea/acessorios-revestimento/chelsea-dark-red/14x26-natura